In [ ]:
from PIL import Image
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import pandas

class FeatureExtractor:
    def __init__(self):
        base_model = ResNet50(weights="/content/drive/MyDrive/crawl_shopee/weight_resnet50.h5", include_top=False)
        x = base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        self.model = tf.keras.Model(inputs=base_model.input, outputs=x)
        
    def extract(self, img):
        
        x = preprocess_input(img)  # Subtracting avg values for each pixel
        feature = self.model.predict(x)  # (1, 2048)
        return feature # Normalize

In [ ]:
def read_image(path, size = (224, 224)):
  # Mở ảnh
  img = np.float32(Image.open(path).resize(size))

  # Sample zeros
  sample = np.zeros((224,224,3))

  # Kiểm tra kích thước ảnh
  if img.shape != sample.shape:
    img = sample
    print(path)
  
  return img

In [ ]:
root_img_path = "/content/drive/MyDrive/crawl_shopee/img"
root_fearure_path = "/content/drive/MyDrive/crawl_shopee/feature/"
data_products_path = "/content/drive/MyDrive/crawl_shopee/data.csv"

size = (224, 224)

fe = FeatureExtractor()

imgs_feature = []

# feature & name_images
x = 0
images = [name for name in os.listdir(root_img_path) if name.endswith((".jpg", ".png", ".jpeg"))]
name_images = []

for i in range(0, len(images) + 1, 348):
  if i == 0: 
    continue
  print(i)
  images_np = []
  for j in range(x, i):
    # Lưu ảnh vào mảng numpy
    images_np.append(read_image(root_img_path + "/" + f"{j}.jpg"))

  images_np_arr = np.array(images_np)
  imgs_feature.extend(fe.extract(images_np_arr))
  x = i

348
696
1044
1392
1740
2088
2436
2784
3132
3480
3828
/content/drive/MyDrive/crawl_shopee/img/3638.jpg
4176
4524
4872
5220
5568
5916
6264
6612
6960
7308
7656
8004
8352
8700
9048
9396
9744
10092
10440
10788
11136
11484
11832
12180
12528
12876
13224
13572
13920
14268
14616
14964
15312
15660
16008
16356
16704
17052
17400
17748
18096
18444
18792
19140
19488
19836
20184
20532
20880
21228
21576
21924
22272
22620
22968
23316
23664
24012
24360
24708


In [ ]:
data = pandas.read_csv(data_products_path, usecols=[0,2,3], header=None)

# Category
category = data.values[:, 0]

# Prices
price = data.values[:, 1]

# Url products
url = data.values[:, 2]

In [ ]:
np.savez_compressed(root_fearure_path + "features_product", 
                    arr1 = imgs_feature, arr2 = images_np_arr, arr3 = category, arr4 = price, arr5 = url)

In [ ]:
# Test read data from features_product

import numpy as np
root_fearure_path = "/content/drive/MyDrive/crawl_shopee/feature/features_product.npz"

data = np.load(root_fearure_path, allow_pickle=True)

imgs_feature = data["arr1"]
name_images = data["arr2"]
categorys = data["arr3"]
prices = data["arr4"]
urls = data["arr5"]